<a href="https://colab.research.google.com/github/ThiagoMSo/Imersao_Alura/blob/main/Assistente-Virtual-para-%C3%81rea-de-Lazer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import numpy as np
import pandas as pd
import json
import random
import markdown
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from google.api_core import retry

GOOGLE_API_KEY="GOOGLE_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

categorias = {
    "valores": ["valor", "preço", "diária", "desconto"],
    "comodidades": ["cômodo", "quarto", "banheiro", "cozinha", "piscina", "churrasqueira"],
    "regras": ["regra", "permitido", "proibido", "som", "limpeza"],
}

def classificar_pergunta(pergunta):
    import spacy
    nlp = spacy.load("en_core_web_sm")

    pergunta_documento = nlp(pergunta)
    entidades = [(ent.text, ent.label_) for ent in pergunta_documento.ents]

    for categoria, palavras_chave in categorias.items():
        for palavra in palavras_chave:
            for entidade in entidades:
                if palavra in entidade[0].lower():
                    return categoria
    return "desconhecido"

data = {
  "nome": "Área de Lazer Marqueti",
  "descricao": "Espaço ideal para eventos e confraternizações, com piscina, área de churrasco e acomodações para até 7 pessoas.",
  "comodidades": [
    {"nome": "Quartos", "quantidade": 2, "descricao": "2 beliches e 1 cama queen size cada"},
    {"nome": "Suítes", "quantidade": 1},
    {"nome": "Banheiros", "quantidade": 3, "descricao": "2 internos + 1 externo"},
    {"nome": "Cozinha", "descricao": "Completa (pia, fogão a gás, geladeira com freezer, micro-ondas, banquetas e armários)"},
    {"nome": "Sala de Jantar", "descricao": "Mesa e 8 cadeiras"},
    {"nome": "Piscina", "descricao": "Adulto com cascata, chuveirão e espreguiçadeiras"},
    {"nome": "Churrasqueira", "descricao": "Área completa (churrasqueira, pia, bancada, freezer, fogão a gás, banquetas de madeira e mesa de bilhar)"},
    {"nome": "Espaço Coberto", "capacidade": 35, "descricao": "Capacidade total: 60 pessoas"},
    {"nome": "Wi-fi", "disponivel": True},
    {"nome": "Televisão", "disponivel": True},
    {"nome": "Som", "disponivel": True},
    {"nome": "Jogos de Luzes", "disponivel": True},
    {"nome": "Mesas e Cadeiras", "quantidade": 7, "descricao": "Jogos de mesas de plástico com cadeiras"},
    {"nome": "Portão Eletrônico", "disponivel": True},
    {"nome": "Estacionamento", "capacidade": "3-4 carros"},
    {"nome": "Mesa de Vidro", "descricao": "Com 4 cadeiras e ombrelone"}
  ],
  "valores": {
    "diaria": 550,
    "desconto": {
      "dias": 3,
      "percentual": 20
    }
  },
  "regras": [
    "Som alto proibido após as 22h (bairro residencial)",
    "Proibido bandas ou som automotivo",
    "Espaço entregue limpo e organizado, devolução no mesmo estado ou taxa de R$ 100 para limpeza",
    "Reserva mediante 30% de sinal na assinatura do contrato e restante na entrega das chaves"
  ],
  "informacoesAdicionais": {
    "eventos": [
      "aniversários",
      "bodas",
      "batizados",
      "chás de bebê",
      "confraternizações",
      "churrascos"
    ],
    "restricoes": [
      "festas universitárias",
      "raves",
      "eventos com fins lucrativos"
    ],
    "lencois": False,
    "piscinaAquecida": False,
    "localizacao": "Bairro Portinari"
  },
  "links": {
    "instagram": "aleatorio",
    "site": "site aleatorio"
  },
  "contato": {
    "telefone": "+55 XX XXXX-XXXX",
    "whatsapp": "+55 XX XXXX-XXXX"
  },
  "contagem_comodos": { # Nova seção
    "quartos": 2,
    "suites": 1,
    "banheiros": 3,
    "cozinha": 1,
    "sala_de_jantar": 1,
    "piscina": 1,
    "churrasqueira": 1,
    "espaco_coberto": 1
  }
}

json_data = json.loads(json.dumps(data))

titulos = []
conteudos = []

titulos.append("Nome")
conteudos.append(json_data["nome"])

titulos.append("Descrição")
conteudos.append(json_data["descricao"])

for comodidade in json_data["comodidades"]:
    titulos.append(comodidade["nome"])
    conteudo = f"{comodidade.get('descricao', '')} "
    if "quantidade" in comodidade:
      conteudo += f"Quantidade: {comodidade['quantidade']}"
    conteudos.append(conteudo)

titulos.append("Valores")
conteudo = f"Diária: R$ {json_data['valores']['diaria']}\n"
conteudo += f"Desconto de {json_data['valores']['desconto']['percentual']}% para locação acima de {json_data['valores']['desconto']['dias']} dias"
conteudos.append(conteudo)

titulos.append("Regras")
conteudo = "\n".join(json_data["regras"])
conteudos.append(conteudo)

titulos.append("Informações Adicionais")
conteudo = f"Eventos: {', '.join(json_data['informacoesAdicionais']['eventos'])}\n"
conteudo += f"Restrições: {', '.join(json_data['informacoesAdicionais']['restricoes'])}\n"
conteudo += f"Lençóis: {'Sim' if json_data['informacoesAdicionais']['lencois'] else 'Não'}\n"
conteudo += f"Piscina Aquecida: {'Sim' if json_data['informacoesAdicionais']['piscinaAquecida'] else 'Não'}\n"
conteudo += f"Localização: {json_data['informacoesAdicionais']['localizacao']}"
conteudos.append(conteudo)

titulos.append("Links")
conteudo = f"Instagram: {json_data['links']['instagram']}\n"
conteudo += f"Site: {json_data['links']['site']}"
conteudos.append(conteudo)

titulos.append("Contato")
conteudo = f"Telefone: {json_data['contato']['telefone']}\n"
conteudo += f"WhatsApp: {json_data['contato']['whatsapp']}"
conteudos.append(conteudo)

df = pd.DataFrame({"Titulo": titulos, "Conteudo": conteudos})
print(df)

model = "models/embedding-001"

def embed_fn(title, text, model):
    return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"], model), axis=1)

def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(base["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return base.iloc[indice]["Conteudo"]

generation_config = {
    "temperature": 0.5,
    "candidate_count": 1
}


saudacoes = [
    " Bom dia! \n \n  Área de Lazer Marqueti, gradece seu contato. \n \n 😊 Procurando um lugar incrível para relaxar?  Como Podemos te Ajudar? \n \n",
    " E aí! 👋 \n \n  Área de Lazer Marqueti, gradece seu contato. \n \n Quer conhecer um espaço perfeito para seus eventos? \n \n",
    "  😄 \n \n Área de Lazer Marqueti, gradece seu contato. \n \n Pronto para descobrir a área de lazer dos seus sonhos?  Como Podemos te Ajudar? \n \n",
    "  😄\n\n Área de Lazer Marqueti, agradece seu contato. \n\n Conte-me: qual o seu sonho para um evento perfeito? ✨ Estou aqui para te ajudar a realizá-lo!  Como Podemos te Ajudar? 😉 \n \n",
    " E aí! 😊\n\n Área de Lazer Marqueti, agradece seu contato. \n\n Que tipo de evento você está organizando? 🎉 Me fale mais detalhes para que eu possa te auxiliar da melhor forma!  Como Podemos te Ajudar? 😄 \n \n",
    " Opa! Tudo bem? 😊\n\n Área de Lazer Marqueti por aqui! \n\n Procurando um lugar bacana para se divertir? 🎉 Conte-me seus planos! 😉 \n \n",
    " 👋\n\n Área de Lazer Marqueti na área! \n\n Chegou a hora de planejar aquele evento especial? 🤩 Vamos juntos encontrar a melhor data para você! Como Podemos te Ajudar? 🥳 \n \n" ,
]


def responder_com_toque_humano(texto, df, model, generation_config, contexto_conversa=None, nome_usuario=None, resposta_contagem=None):
    resposta_embedding = gerar_e_buscar_consulta(texto, df, model)

    # Construção do prompt com contexto e informações do usuário
    prompt = f"""
Contexto da conversa: {contexto_conversa if contexto_conversa else "Primeira interação"}
Nome do usuário: {nome_usuario if nome_usuario else "Usuário"}
Tom desejado: Amigável e informativo.
Informação relevante do JSON: {resposta_final}
Pergunta do usuário: {consulta}

Responda à pergunta do usuário de forma concisa e direta, utilizando as informações do JSON fornecidas. Se a pergunta for sobre valores, forneça a informação exata. Se a pergunta for sobre comodidades, descreva a comodidade e seus detalhes.
Se a pergunta for sobre regras, cite a regra específica.
"""

    model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
    try:
        response = model_2.generate_content(prompt)
        if response.candidates and response.candidates[0].content.parts:
            resposta_markdown = response.candidates[0].content.parts[0].text
        else:
            resposta_markdown = "Desculpe, não consegui entender a sua pergunta. Poderia reformular?"
    except ValueError:
        resposta_markdown = "Desculpe, não consegui processar sua pergunta no momento. Tente novamente mais tarde."


    frases_positivas = ["\n Ótima pergunta! \n", "Que legal! 😄 \n"]
    resposta_markdown = random.choice(frases_positivas) + resposta_markdown

    emojis = ["😊", "😄", "😎", "🥳", "🏊"]
    resposta_markdown += " " + random.choice(emojis)

    resposta_html = markdown.markdown(resposta_markdown)

    return resposta_html, resposta_markdown

df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"], model), axis=1)

primeira_interacao = True
contexto_conversa = ""
nome_usuario = ""
nome_usuario = input("Olá! 😊 Qual é o seu nome? ")

while True:
    if primeira_interacao:
        consulta = input(f"\n Prazer em te conhecer, {nome_usuario}! {random.choice(saudacoes)}")
        primeira_interacao = False
    else:
        consulta = input("\n O que você gostaria de saber? 🤔 \n")

    categoria = classificar_pergunta(consulta)

    if categoria == "comodidades" and "quantos comodos" in consulta.lower():
        resposta_final = f"A casa possui {json_data['contagem_comodos']['quartos']} quartos, {json_data['contagem_comodos']['suites']} suítes, {json_data['contagem_comodos']['banheiros']} banheiros, {json_data['contagem_comodos']['cozinha']} cozinha, {json_data['contagem_comodos']['sala_de_jantar']} sala de jantar, {json_data['contagem_comodos']['piscina']} piscina, {json_data['contagem_comodos']['churrasqueira']} churrasqueira e {json_data['contagem_comodos']['espaco_coberto']} espaço coberto."
        resposta_html, resposta_markdown = responder_com_toque_humano(consulta, df, model, generation_config, contexto_conversa, nome_usuario, resposta_contagem=resposta_final)
    else:
        if categoria != "desconhecido":
            df_filtrado = df[df["Titulo"].str.lower().str.contains("|".join(categorias[categoria]))]
            resposta_embedding = gerar_e_buscar_consulta(consulta, df_filtrado, model)
        else:
            resposta_embedding = gerar_e_buscar_consulta(consulta, df, model)

        resposta_final = resposta_embedding
        resposta_html, resposta_markdown = responder_com_toque_humano(consulta, df, model, generation_config, contexto_conversa, nome_usuario, resposta_contagem=resposta_final)

        if consulta.lower() in ["sair", "tchau", "até mais", "adeus", "flw", "falou", "vlw", "obrigado", "obrigada"]:
          print(random.choice([
            "Foi um prazer conversar com você! Até mais! 👋",
            "Tchau! 😊 Até a próxima!",
            "Obrigado por conversar comigo! 😄",
            "Espero ter ajudado! 😉 Até logo!",
        ]))
          break
        else:
          contexto_conversa += f"\nUsuário: {consulta}\n"

        display(Markdown(resposta_html))

        contexto_conversa += f"Chatbot: {resposta_markdown}\n"

models/embedding-001
models/text-embedding-004
                    Titulo                                           Conteudo
0                     Nome                             Área de Lazer Marqueti
1                Descrição  Espaço ideal para eventos e confraternizações,...
2                  Quartos  2 beliches e 1 cama queen size cada Quantidade: 2
3                   Suítes                                      Quantidade: 1
4                Banheiros               2 internos + 1 externo Quantidade: 3
5                  Cozinha  Completa (pia, fogão a gás, geladeira com free...
6           Sala de Jantar                                 Mesa e 8 cadeiras 
7                  Piscina  Adulto com cascata, chuveirão e espreguiçadeiras 
8            Churrasqueira  Área completa (churrasqueira, pia, bancada, fr...
9           Espaço Coberto                      Capacidade total: 60 pessoas 
10                   Wi-fi                                                   
11               

<p>Ótima pergunta! 
A casa tem 2 quartos. Cada quarto possui 2 beliches e 1 cama queen size. 😎</p>


 O que você gostaria de saber? 🤔 
Qual valor do aluguel?


<p>Que legal! 😄 
O valor do aluguel é de R$ 550 por dia. Se você alugar por mais de 3 dias, receberá um desconto de 20%. 😄</p>


 O que você gostaria de saber? 🤔 
Qual valor da diaria?


...

<p>Que legal! 😄 
Que legal! 😄 </p>
<p>Nossa área completa oferece:</p>
<ul>
<li>Churrasqueira</li>
<li>Pia</li>
<li>Bancada</li>
<li>Freezer</li>
<li>Fogão a gás</li>
<li>Banquetas de madeira</li>
<li>Mesa de bilhar 😄</li>
</ul>


 O que você gostaria de saber? 🤔 
Qual telefone da area?


<p>Que legal! 😄 
Que legal! 😄 </p>
<p>O telefone da área é: +55 XX XXXX-XXXX. 😄</p>


 O que você gostaria de saber? 🤔 
Poderia me mandar alguma foto da area?


<p>Ótima pergunta! 
Que legal! 😄 </p>
<p>Não possuo fotos da área no momento. 😎</p>